In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1"

import numpy as np, gc, re
import pandas as pd

# train = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv")
# print("Train shape",train.shape)
# display(train.head())
# print()

# test = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv")
# print("Test shape",test.shape)
# display(test.head())

In [4]:
from transformers import AutoModel,AutoTokenizer
import torch, torch.nn.functional as F
from tqdm import tqdm

In [ ]:
# def mean_pooling(model_output, attention_mask):
#     token_embeddings = model_output.last_hidden_state.detach().cpu()
#     input_mask_expanded = (
#         attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
#     )
#     return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
#         input_mask_expanded.sum(1), min=1e-9
#     )

In [ ]:
# class EmbedDataset(torch.utils.data.Dataset):
#     def __init__(self,df,tokenizer,max_length):
#         self.df = df.reset_index(drop=True)
#         self.tokenizer = tokenizer
#         self.max = max_length
#     def __len__(self):
#         return len(self.df)
#     def __getitem__(self,idx):
#         text = self.df.loc[idx,"full_text"]
#         tokens = self.tokenizer(
#                 text,
#                 None,
#                 add_special_tokens=True,
#                 padding='max_length',
#                 truncation=True,
#                 max_length=self.max,
#                 return_tensors="pt")
#         tokens = {k:v.squeeze(0) for k,v in tokens.items()}
#         return tokens

In [ ]:
# from transformers import AutoModel, LongformerTokenizer, LEDTokenizer, BartTokenizer, DebertaTokenizer, DebertaV2Tokenizer, BigBirdTokenizer

# def get_embeddings(model_name='', max_length=1024, batch_size=32, compute_train=True, compute_test=True):
#     global train, test
#     DEVICE = "cuda:1" # EXTRACT EMBEDDINGS WITH GPU #2
#     path = "/kaggle/input/huggingface1024models/"
#     disk_name = path + model_name.replace("/","_")


#     model = AutoModel.from_pretrained(disk_name, trust_remote_code=True)

#     # Use specific tokenizer classes
#     if 'led' in model_name.lower():
#         tokenizer = LEDTokenizer.from_pretrained(disk_name, trust_remote_code=True)
#     elif 'longformer' in model_name.lower():
#         tokenizer = LongformerTokenizer.from_pretrained(disk_name, trust_remote_code=True)
#     elif 'bart' in model_name.lower():
#         tokenizer = BartTokenizer.from_pretrained(disk_name, trust_remote_code=True)
#     elif 'deberta-v3' in model_name.lower():
#         tokenizer = DebertaV2Tokenizer.from_pretrained(disk_name, trust_remote_code=True)
#     elif 'deberta' in model_name.lower():
#         tokenizer = DebertaTokenizer.from_pretrained(disk_name, trust_remote_code=True)
#     elif 'bigbird' in model_name.lower():
#         tokenizer = BigBirdTokenizer.from_pretrained(disk_name, trust_remote_code=True)
#     else:
#         raise ValueError(f"Unknown model type for {model_name}")

#     ds_tr = EmbedDataset(train, tokenizer, max_length)
#     embed_dataloader_tr = torch.utils.data.DataLoader(ds_tr,
#                             batch_size=batch_size,
#                             shuffle=False)
#     ds_te = EmbedDataset(test, tokenizer, max_length)
#     embed_dataloader_te = torch.utils.data.DataLoader(ds_te,
#                             batch_size=batch_size,
#                             shuffle=False)

#     model = model.to(DEVICE)
#     model.eval()

#     # COMPUTE TRAIN EMBEDDINGS
#     all_train_text_feats = []
#     if compute_train:
#         for batch in tqdm(embed_dataloader_tr,total=len(embed_dataloader_tr)):
#             input_ids = batch["input_ids"].to(DEVICE)
#             attention_mask = batch["attention_mask"].to(DEVICE)
#             with torch.no_grad():
#                 with torch.cuda.amp.autocast(enabled=True):
#                     model_output = model(input_ids=input_ids,attention_mask=attention_mask)
#             sentence_embeddings = mean_pooling(model_output, attention_mask.detach().cpu())
#             # Normalize the embeddings
#             sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
#             sentence_embeddings =  sentence_embeddings.squeeze(0).detach().cpu().numpy()
#             all_train_text_feats.extend(sentence_embeddings)
#     all_train_text_feats = np.array(all_train_text_feats)

#     # COMPUTE TEST EMBEDDINGS
#     all_test_text_feats = []
#     if compute_test:
#         for batch in embed_dataloader_te:
#             input_ids = batch["input_ids"].to(DEVICE)
#             attention_mask = batch["attention_mask"].to(DEVICE)
#             with torch.no_grad():
#                 with torch.cuda.amp.autocast(enabled=True):
#                     model_output = model(input_ids=input_ids,attention_mask=attention_mask)
#             sentence_embeddings = mean_pooling(model_output, attention_mask.detach().cpu())
#             # Normalize the embeddings
#             sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
#             sentence_embeddings =  sentence_embeddings.squeeze(0).detach().cpu().numpy()
#             all_test_text_feats.extend(sentence_embeddings)
#         all_test_text_feats = np.array(all_test_text_feats)
#     all_test_text_feats = np.array(all_test_text_feats)

#     # CLEAR MEMORY
#     del ds_tr, ds_te
#     del embed_dataloader_tr, embed_dataloader_te
#     del model, tokenizer
#     del model_output, sentence_embeddings, input_ids, attention_mask
#     gc.collect()
#     torch.cuda.empty_cache()

#     # RETURN EMBEDDINGS
#     return all_train_text_feats, all_test_text_feats

In [ ]:
# models = [
#     ('microsoft/deberta-base', 1024, 32),
#     ('microsoft/deberta-large', 1024, 8),
#     ('microsoft/deberta-v3-large', 1024, 8),
#     ('allenai/longformer-base-4096', 1024, 32),
#     ('google/bigbird-roberta-base', 1024, 32),
#     ('google/bigbird-roberta-large', 1024, 8),
#     ('allenai/led-base-16384', 1024, 32),
#     ('allenai/led-large-16384', 1024, 8),
#     ("facebook/bart-large", 1024, 8),
#     ('allenai/longformer-large-4096', 1024, 8),
# ]

In [5]:
# path = "/kaggle/input/essay-embeddings/"
# all_train_embeds = []
# all_test_embeds = []

# for (model, max_length, batch_size) in models:
#     name = path + model.replace("/","_") + ".npy"
#     if os.path.exists(name):
#         _, test_embed = get_embeddings(model_name=model, max_length=max_length, batch_size=batch_size, compute_train=False)
#         train_embed = np.load(name)
#         print(f"Loading train embeddings for {name}")
#     else:
#         print(f"Computing train embeddings for {name}")
#         train_embed, test_embed = get_embeddings(model_name=model, max_length=max_length, batch_size=batch_size, compute_train=True)
#         np.save(name, train_embed)
#     all_train_embeds.append(train_embed)
#     all_test_embeds.append(test_embed)

# del train_embed, test_embed

In [8]:
all_train_embeds = np.load('/content/drive/MyDrive/all_train_embeds.npy')
all_test_embeds = np.load('/content/drive/MyDrive/all_test_embeds.npy')

gc.collect()
print('Our concatenated train embeddings have shape', all_train_embeds.shape )


Our concatenated train embeddings have shape (17307, 9216)


In [5]:
proprocessed_features = pd.read_csv('/content/drive/MyDrive/preprocessed_features .csv')
#proprocessed_features = pd.concat([proprocessed_features, pd.DataFrame(all_train_embeds)], axis=1)

In [7]:
proprocessed_features.head()

,essay_id,word_12_cnt,word_14_cnt,word_len_max,word_len_std,sentence_150_cnt,sentence_250_cnt,sentence_word_cnt_max,sentence_len_min,paragraph_100_cnt,...,ts_ccomp,ts_compound,ts_csubj,ts_neg,ts_nmod,ts_nsubjpass,ts_oprd,ts_pcomp,ts_relcl,ts_xcomp
0,000d118,6,5,25,2.538495,7,3,126,36,1,...,19,22,1,6,3,7,2,3,12,6
1,000fe60,0,0,11,2.060968,2,0,48,26,5,...,11,3,1,6,0,1,0,2,10,2
2,001ab80,14,5,15,2.604621,7,0,46,58,4,...,11,8,1,12,2,2,0,6,12,13
3,001bdc0,9,1,17,2.767791,6,3,64,25,5,...,11,12,1,2,0,2,0,5,3,2
4,002ba53,9,2,14,2.861249,6,2,88,17,4,...,3,19,1,3,1,4,0,9,9,2


In [8]:
proprocessed_features = proprocessed_features.drop(proprocessed_features.filter(regex='^tfidf_').columns, axis=1)

In [11]:
proprocessed_features.to_csv('proprocessed_features_fin.csv')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
proprocessed_features = proprocessed_features.drop(['essay_id'],axis=1)

In [13]:
all_train_embeds = np.array(proprocessed_features)
all_test_embeds  = all_train_embeds[:3]

In [14]:
train = pd.read_csv('/content/drive/MyDrive/train.csv')
test = pd.read_csv('/content/drive/MyDrive/test.csv')

In [15]:
def find_thresholds(true, pred, steps=50):

    # SAVE TRIALS FOR PLOTTING
    xs = [[],[],[],[],[]]
    ys = [[],[],[],[],[]]

    # COMPUTE BASELINE METRIC
    threshold = [1.5, 2.5, 3.5, 4.5, 5.5]
    pred2 = pd.cut(pred, [-np.inf] + threshold + [np.inf],
                    labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred2, weights="quadratic")

    # FIND FIVE OPTIMAL THRESHOLDS
    for k in range(5):
        for sign in [1,-1]:
            v = threshold[k]
            threshold2 = threshold.copy()
            stop = 0
            while stop<steps:

                # TRY NEW THRESHOLD
                v += sign * 0.001
                threshold2[k] = v
                pred2 = pd.cut(pred, [-np.inf] + threshold2 + [np.inf],
                                labels=[1,2,3,4,5,6]).astype('int32')
                metric = cohen_kappa_score(true, pred2, weights="quadratic")

                # SAVE TRIALS FOR PLOTTING
                xs[k].append(v)
                ys[k].append(metric)

                # EARLY STOPPING
                if metric<=best:
                    stop += 1
                else:
                    stop = 0
                    best = metric
                    threshold = threshold2.copy()

    # COMPUTE FINAL METRIC
    pred2 = pd.cut(pred, [-np.inf] + threshold + [np.inf],
                    labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred2, weights="quadratic")

    # RETURN RESULTS
    threshold = [np.round(t,3) for t in threshold]
    return best, threshold, xs, ys

In [16]:
from sklearn.model_selection import StratifiedKFold
FOLDS = 3
train_xgb = train.copy()
train_xgb["fold"] = -1
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
for fold, (train_index, val_index) in enumerate(skf.split(train_xgb, train_xgb["score"])):
    train_xgb.loc[val_index, "fold"] = fold
print('Train samples per fold:')
print(train_xgb.fold.value_counts().sort_index())


Train samples per fold:
0    5769
1    5769
2    5769
Name: fold, dtype: int64


In [18]:
import numpy as np
import pandas as pd
from sklearn.metrics import cohen_kappa_score
import xgboost as xgb
from tqdm import tqdm

def comp_score(y_true, y_pred):
    p = y_pred.clip(1, 6).round(0)
    m = cohen_kappa_score(y_true, p, weights='quadratic')
    return m

In [19]:


xgb_params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'booster': 'gbtree',
    'max_depth': 6,
    'learning_rate': 0.01,
    'subsample': 0.8,
    'colsample_bytree': 0.6,
    'colsample_bylevel': 0.6,
    'n_estimators': 10000,
    'device': 'cuda',
    'early_stopping_rounds': 100,
    'gamma': 0.1,
    'min_child_weight': 1,
    'reg_alpha': 0.1,
    'reg_lambda': 1,
}

xgb_oof = np.zeros(len(train_xgb), dtype='float32')
xgb_test_preds = np.zeros((len(test), FOLDS), dtype='float32')

for fold in range(FOLDS):
    print('#'*25)
    print('### Fold', fold+1)
    print('#'*25)

    train_index = train_xgb["fold"] != fold
    valid_index = train_xgb["fold"] == fold

    X_train = all_train_embeds[train_index]
    y_train = train_xgb.loc[train_index, 'score'].values
    X_valid = all_train_embeds[valid_index]
    y_valid = train_xgb.loc[valid_index, 'score'].values
    X_test = all_test_embeds

    xgb_model = xgb.XGBRegressor(**xgb_params)
    xgb_model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=100
    )

    xgb_preds = xgb_model.predict(X_valid)
    xgb_test_preds[:, fold] = xgb_model.predict(X_test)
    xgb_oof[valid_index] = xgb_preds

    score = comp_score(y_valid, xgb_preds)
    print(f"=> QWK score: {score}")
    print()

print('#'*25)
xgb_score = comp_score(train_xgb['score'].values, xgb_oof)
print('Overall CV QWK score before threshold optimization =', xgb_score)

#########################
### Fold 1
#########################
[0]	validation_0-rmse:1.03839
[100]	validation_0-rmse:0.68345
[200]	validation_0-rmse:0.58918
[300]	validation_0-rmse:0.56132
[400]	validation_0-rmse:0.55063
[500]	validation_0-rmse:0.54545
[600]	validation_0-rmse:0.54222
[700]	validation_0-rmse:0.54004
[800]	validation_0-rmse:0.53853
[900]	validation_0-rmse:0.53743
[1000]	validation_0-rmse:0.53672
[1100]	validation_0-rmse:0.53605
[1200]	validation_0-rmse:0.53544
[1300]	validation_0-rmse:0.53503
[1400]	validation_0-rmse:0.53457
[1500]	validation_0-rmse:0.53416
[1600]	validation_0-rmse:0.53391
[1700]	validation_0-rmse:0.53376
[1800]	validation_0-rmse:0.53349
[1900]	validation_0-rmse:0.53328
[2000]	validation_0-rmse:0.53306
[2100]	validation_0-rmse:0.53295
[2200]	validation_0-rmse:0.53279
[2300]	validation_0-rmse:0.53266
[2400]	validation_0-rmse:0.53253
[2500]	validation_0-rmse:0.53246
[2600]	validation_0-rmse:0.53241
[2700]	validation_0-rmse:0.53231
[2800]	validation_0-rmse:

In [20]:
print('#'*25)
xgb_score = comp_score(train_xgb['score'].values, xgb_oof)
print('Overall CV QWK score before threshold optimization =', xgb_score)

# Threshold optimization
xgb_best_score, xgb_best_thresholds, xs, ys = find_thresholds(train['score'].values, xgb_oof)

print(f"Best QWK score after threshold optimization: {xgb_best_score}")
print(f"Best thresholds: {xgb_best_thresholds}")

# Apply best thresholds to predictions
xgb_oof_optimized = pd.cut(xgb_oof, [-np.inf] + xgb_best_thresholds + [np.inf],
                           labels=[1,2,3,4,5,6]).astype('int32')

# Calculate final QWK score
xgb_final_score = comp_score(train['score'].values, xgb_oof_optimized)
print(f"Final QWK score after threshold optimization: {xgb_final_score}")



#########################
Overall CV QWK score before threshold optimization = 0.8159241311560093
Best QWK score after threshold optimization: 0.8320088056760485
Best thresholds: [1.73, 2.491, 3.434, 4.28, 5.172]
Final QWK score after threshold optimization: 0.8320088056760485


In [2]:
!pip uninstall -y cupy-cuda12x
!pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12 cuml-cu12
!pip install pandas==1.5.3

Found existing installation: cupy-cuda12x 12.2.0
Uninstalling cupy-cuda12x-12.2.0:
  Successfully uninstalled cupy-cuda12x-12.2.0
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 478.0/478.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 833.8/833.8 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.9/186.9 MB 9.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of cuml-cu12 to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.4 MB/s eta 0:00:00
     ━━━━

In [3]:
from cuml.svm import SVR
import cuml
print('RAPIDS version',cuml.__version__)

RAPIDS version 24.04.00


In [23]:
from sklearn.metrics import cohen_kappa_score

svr_oof = np.zeros(len(train_svr), dtype='float32')
svr_test_preds = np.zeros((len(test),FOLDS), dtype='float32')

def comp_score(y_true, y_pred):
    p = y_pred.clip(1, 6).round(0)
    m = cohen_kappa_score(y_true, p, weights='quadratic')
    return m

In [24]:
FOLDS = 15
train_svr = train.copy()
train_svr["fold"] = -1
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
for fold,(train_index, val_index) in enumerate(skf.split(train_svr, train_svr["score"])):
    train_svr.loc[val_index,"fold"] = fold
print('Train samples per fold:')
train_svr.fold.value_counts().sort_index()

Train samples per fold:


0     1154
1     1154
2     1154
3     1154
4     1154
5     1154
6     1154
7     1154
8     1154
9     1154
10    1154
11    1154
12    1153
13    1153
14    1153
Name: fold, dtype: int64

In [32]:
for fold in range(FOLDS):
    print('#'*25)
    print('### Fold',fold+1)
    print('#'*25)

    train_index = train_svr["fold"] != fold
    valid_index = train_svr["fold"] == fold

    X_train = all_train_embeds[train_index,].astype(np.float32) # Convert to float32
    y_train = train_svr.loc[train_index,'score'].values.astype(np.float32) # Convert to float32
    X_valid = all_train_embeds[valid_index,].astype(np.float32) # Convert to float32
    y_valid = train_svr.loc[valid_index,'score'].values.astype(np.float32) # Convert to float32
    X_test = all_test_embeds.astype(np.float32) # Convert to float32

    model = SVR(C=10)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    svr_test_preds[:,fold] = model.predict(X_test)
    svr_oof[valid_index] = preds

    score = comp_score(y_valid, preds)
    print(f"=> QWK score: {score}")
    print()

#########################
### Fold 1
#########################


RuntimeError: exception occurred! file=/__w/cuml/cuml/cpp/src/svm/smosolver.cuh line=541: SMO error: NaN found during fitting. This might be caused by floating point overflow. In such case using fp64 could help. Alternatively, try gamma='scale' kernel parameter.
Obtained 63 stack frames
#1 in /usr/local/lib/python3.10/dist-packages/cuml/internals/../libcuml++.so: ML::SVM::SmoSolver<float>::CheckStoppingCondition(float) +0x40c [0x7d0650d8f96c]
#2 in /usr/local/lib/python3.10/dist-packages/cuml/internals/../libcuml++.so: void ML::SVM::SmoSolver<float>::Solve<std::experimental::mdspan<float, std::experimental::extents<int, 18446744073709551615ul, 18446744073709551615ul>, std::experimental::layout_stride, raft::host_device_accessor<std::experimental::default_accessor<float>, (raft::memory_type)2> > >(std::experimental::mdspan<float, std::experimental::extents<int, 18446744073709551615ul, 18446744073709551615ul>, std::experimental::layout_stride, raft::host_device_accessor<std::experimental

In [ ]:
print('#'*25)
svr_score = comp_score(train['score'].values, svr_oof)
print('Overall CV QWK score before threshold optimization =', svr_score)

# Threshold optimization
svr_best_score, svr_best_thresholds, xs, ys = find_thresholds(train['score'].values, svr_oof)

print(f"Best QWK score after threshold optimization: {svr_best_score}")
print(f"Best thresholds: {svr_best_thresholds}")

# Apply best thresholds to predictions
svr_oof_optimized = pd.cut(svr_oof, [-np.inf] + svr_best_thresholds + [np.inf],
                           labels=[1,2,3,4,5,6]).astype('int32')

# Calculate final QWK score
svr_final_score = comp_score(train['score'].values, svr_oof_optimized)
print(f"Final QWK score after threshold optimization: {svr_final_score}")

#########################
Overall CV QWK score before threshold optimization = 0.8218419185595989
Best QWK score after threshold optimization: 0.83102679623669
Best thresholds: [1.732, 2.592, 3.434, 4.3, 5.417]
Final QWK score after threshold optimization: 0.83102679623669


In [ ]:
xgb_test_preds,svr_test_preds

(array([[2.7453802, 2.7820837, 2.2976243],
        [2.9664438, 2.9593594, 2.7425377],
        [4.1339874, 4.0574946, 4.113041 ]], dtype=float32),
 array([[2.45069  , 2.4886885, 2.427023 , 2.4596071, 2.3950753, 2.4675193,
         2.5393143, 2.458449 , 2.4517207, 2.4911501, 2.325545 , 2.4548576,
         2.39111  , 2.426531 , 2.3667865],
        [2.958594 , 2.8996382, 2.9655972, 2.9468203, 2.9426217, 2.9436545,
         3.0202103, 2.8987565, 2.934936 , 2.9497073, 2.9214313, 2.9229515,
         2.9414487, 2.9191153, 2.8992572],
        [4.3871827, 4.363261 , 4.389578 , 4.4892397, 4.3817697, 4.401754 ,
         4.4463425, 4.4308   , 4.485107 , 4.4162602, 4.460799 , 4.3851094,
         4.3679166, 4.4593983, 4.421779 ]], dtype=float32))

In [ ]:
xgb_test_preds_avg = np.mean(xgb_test_preds, axis=1)
xgb_test_preds_thresholded = pd.cut(xgb_test_preds_avg,
                                    [-np.inf] + xgb_best_thresholds + [np.inf],
                                    labels=[1, 2, 3, 4, 5, 6]).astype(float)

# Apply thresholds to SVR test predictions
svr_test_preds_avg = np.mean(svr_test_preds, axis=1)
svr_test_preds_thresholded = pd.cut(svr_test_preds_avg,
                                    [-np.inf] + svr_best_thresholds + [np.inf],
                                    labels=[1, 2, 3, 4, 5, 6]).astype(float)


In [ ]:
combined_test_preds = (xgb_test_preds_thresholded + svr_test_preds_thresholded) / 2

xgb_svr_test_preds_optimized = np.clip(np.round(combined_test_preds), 1, 6).astype(int)

In [ ]:
sub = pd.read_csv("/content/drive/MyDrive/sample_submission.csv")
sub.head()


,essay_id,score
0,000d118,3
1,000fe60,3
2,001ab80,4


In [ ]:
sub["score"] = xgb_svr_test_preds_optimized
sub.score = sub.score.astype('int32')
sub.to_csv("submission.csv",index=False)
print("Submission shape", sub.shape )
sub.head()

Submission shape (3, 2)


,essay_id,score
0,000d118,2
1,000fe60,3
2,001ab80,4
